In [1]:
import pandas as pd
import pickle
import os

In [2]:
pred_df = pickle.load(open("../output/cv/pred_df.pkl", "rb"))

In [6]:
pred_df[pred_df["session"] == "14267144"]

,session,labels,type
1367365,14267144,[],clicks
3039168,14267144,[],orders
4710971,14267144,[],carts


In [3]:
pred_df = pred_df.explode("labels")

In [4]:
pred_df.head()

,session,labels,type
0,12899779,1253524,clicks
0,12899779,737445,clicks
0,12899779,438191,clicks
0,12899779,731692,clicks
0,12899779,1790770,clicks


In [5]:
pred_df = pred_df.rename(columns={"labels": "aid"})

In [6]:
pred_df.to_csv("pred_df_test.csv", index=False)

In [20]:
df = pd.read_csv("../output/splendid-flower-28/cv/full_columns_validation_preds.csv")
df["session"] = df["session_type"].apply(lambda x: x.split("_")[0])
df["type"] = df["session_type"].apply(lambda x: x.split("_")[1])
df = df[["session", "type", "top_n"]]
df["top_n"] = df["top_n"].apply(lambda x: [int(v) for v in x.split(" ")])
df = df.explode("top_n")
df = df.rename(columns={"top_n": "aid"})
df.head()

,session,type,aid
0,11098528,clicks,11830
0,11098528,clicks,588923
0,11098528,clicks,1732105
0,11098528,clicks,571762
0,11098528,clicks,884502


In [21]:
df.shape

(100348921, 3)

In [22]:
df.to_csv("covisit.csv", index=False)

In [5]:
top_n_clicks = pickle.load(open(os.path.join("../output/splendid-flower-28/cv", f"top_20_clicks_0.pkl"), "rb"))

In [11]:
top_n_clicks[0]

[532042,
 643097,
 1848174,
 1735605,
 1363081,
 1012453,
 251581,
 1256426,
 994505,
 150507,
 39551,
 66456,
 215649,
 285817,
 319593,
 353304,
 435875,
 472412,
 505086,
 878565]

In [1]:
import pandas as pd
import gc
import pickle
import glob
from lightgbm.sklearn import LGBMRanker


def read_files(path):
    dfs = []
    for file in glob.glob(path):
        df = pd.read_parquet(file)
        df["aid"] = df["aid"].astype(int)
        dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)


def read_train_labels():
    train_labels = pd.read_parquet("../input/otto-validation/test_labels.parquet")
    train_labels = train_labels.explode("ground_truth")
    train_labels["aid"] = train_labels["ground_truth"]
    train_labels = train_labels[["session", "type", "aid"]]
    train_labels["aid"] = train_labels["aid"].astype(int)
    return train_labels


def dump_pickle(path, o):
    with open(path, "wb") as f:
        pickle.dump(o, f)


In [2]:
train_all = read_files("../input/lgbm_dataset/*")
print("read train files")
train_labels_all = read_train_labels()

read train files


In [3]:
train_all.shape

(49740239, 28)

In [6]:
train_all["session_length"][:30]

0     30
1     28
2     28
3     28
4     28
5     28
6     28
7     28
8     28
9     28
10    28
11    28
12    28
13    28
14    28
15    28
16    28
17    28
18    28
19    28
20    28
21    28
22    28
23    28
24    28
25    28
26    28
27    28
28    28
29    29
Name: session_length, dtype: int64

In [5]:
type = "clicks"

In [6]:
print(f"type={type} start")
train_labels = train_labels_all[train_labels_all["type"] == type]
train_labels["gt"] = 1
train = train_all.merge(train_labels, how="left", on=["session", "aid"])
del train_labels
gc.collect()
train["gt"].fillna(0, inplace=True)
train["gt"] = train["gt"].astype(int)
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=20,
    importance_type="gain",
)
target = "gt"
feature_cols = train.drop(columns=[target, "session", "type"]).columns.tolist()
# train
print(f"train shape: {train.shape}")
print(f"group sum: {train['session_length'].sum()}")

type=clicks start


/tmp/ipykernel_52/1721257436.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_labels["gt"] = 1


train shape: (49740239, 30)
group sum: 1445094293


In [7]:
train.head()

,session,aid,session_interaction_length,session_length,clicks_cnt,carts_cnt,orders_cnt,this_aid_clicks_cnt,this_aid_carts_cnt,this_aid_orders_cnt,...,min_log_recency_score,max_log_recency_score,avg_type_weighted_log_recency_score,min_type_weighted_log_recency_score,max_type_weighted_log_recency_score,covisit_clicks_candidate_num,covisit_carts_candidate_num,covisit_orders_candidate_num,type,gt
0,12245144,1791715,3,30,1,2,0,0.0,1.0,0.0,...,0.464086,0.464086,2.784514,2.784514,2.784514,NaN,NaN,NaN,NaN,0
1,12245145,29735,6,28,6,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,14.0,9.0,17.0,NaN,0
2,12245145,137514,6,28,6,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.0,13.0,NaN,0
3,12245145,177958,6,28,6,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.0,4.0,19.0,clicks,1
4,12245145,368495,6,28,6,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.0,15.0,15.0,NaN,0


In [9]:
train[train["session"] == 12245144].shape

(30, 30)

In [10]:
train = train.sort_values(["session", "aid"])

In [11]:
train.head()

,session,aid,session_interaction_length,session_length,clicks_cnt,carts_cnt,orders_cnt,this_aid_clicks_cnt,this_aid_carts_cnt,this_aid_orders_cnt,...,min_log_recency_score,max_log_recency_score,avg_type_weighted_log_recency_score,min_type_weighted_log_recency_score,max_type_weighted_log_recency_score,covisit_clicks_candidate_num,covisit_carts_candidate_num,covisit_orders_candidate_num,type,gt
7771759,11098528,11830,1,21,1,0,0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,0
7771760,11098528,77440,1,21,1,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,2.0,3.0,NaN,0
7771761,11098528,205357,1,21,1,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,0
7771762,11098528,231487,1,21,1,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,14.0,15.0,NaN,0
7771763,11098528,307904,1,21,1,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,14.0,1.0,2.0,NaN,0


In [ ]:
ranker = ranker.fit(
    train[feature_cols],
    train[target],
    group=train["session_length"].tolist(),
)